### Oscillator-based TRNGs
<p>The first generator to consider is the simplest Oscillator-based generator: the elementary oscillator-based TRNG (EO-TRNG) presented in Fig. 1. </p>
<p align="center" width="100%">
    <img width="50%" src="./eo_trng.PNG"> 
</p>
<p style="text-align: center;"><b>Figure 1: Randomness extraction from a jittered clock signal by a D flip-flop</b></p>
<p>Clearly, the output randomness depends on the following parameters:
    <ol>
      <li>The average clock periods of $O_{0}$ and $O_{1}$: $T_{0}$ and $T_{1}$</li>
      <li>The $O_{1} $duty cycle: $\alpha_1$</li>
      <li>The jitter variance of $O_{0}$ and $O_{1}$: $\sigma_{tot}^2$ </li>
      <li>The accumulation time: $\Delta t = D\cdot T_0$</li>        
    </ol>
    
### The exercice
<p> In this exercise, you can analyze the impact of input parameters of the EO-TRNG on the generator output. 
You can execute the last cell of this notebook and observe how the duty cycle, the jitter variance, and the accumulation time affect the randomness of the generated numbers.</p>

<p> By default $T_{0}=1.875$ ns and  $T_{1}=3.125$ ns, which respectivly correspond to a 3 and a 5 delay based oscillator built with the same inverter based $312.5$ ps delay. $T_{0}$ and  $T_{1}$ can be changed by setting them to the desired values in the next cell of this notebook</p> 

<p> Randomness is analyzed by observing the distribution and the auto-correlation of generated 4-bit bit-vectors.
Note: In the case of an ideal RNG, the distribution and the auto-correlation are in 99.9~\% of the cases between the red lines.</p>


### Your job
<p>The Graal of a TRNG designer is to find the smallest TRNG, generating random numbers with the highest throuput.
For this reason, your goal is to find the smallest $D$ for which the EO-TRNG would not be distinguishable from an ideal RNG.</p>

#### your 1<sup>st</sup> mission

<ul>
  <li>Let fix $\alpha_{1}$ to 0.5 wich correspond to the ideal oscilators duty cycle </li>
  <li>Let fix $\frac{\sigma_{tot}}{T_{1}}$ to 0.001 wich is the most relevant jitter value observed with inverters based oscilators implemented in FPGA's [Ref to be completed]</li>
</ul>
<ul>
<li style="color:green">Your goal for this mission is to determine the smallest $D$ for which the distribution and the auto-correlation would not distinguish the EO-TRNG  from an ideal RNG.</li>
</ul>

#### your 2<sup>nd</sup> mission

<ul>
  <li>Let fix $\alpha_1$ to 0.5 wich correspond to the ideal oscilators duty cycle </li>
</ul>
<ul>
<li><p style="color:green">Your goal for this mission is to determine the smallest $D$ for which the distribution and the auto-correlation would not distinguish the EO-TRNG from an ideal RNG for: </p>
    <ul>
      <li>$\frac{\sigma_{tot}}{T_{1}}=0.0001$ </li>
      <li>$\frac{\sigma_{tot}}{T_{1}}=0.0005$ </li>
      <li>$\frac{\sigma_{tot}}{T_{1}}=0.005$ </li>
      <li>$\frac{\sigma_{tot}}{T_{1}}=0.01$ </li>
    </ul>
</ul>

### 1<sup>st</sup>  conclusion
#### <p style="color:red"> How the jitter variance impact the throughput of a EO-TRNG undistinguishable from an ideal RNG ?</p>

#### your 3<sup>rd</sup> mission
<ul>
  <li>Let fix $\frac{\sigma_{tot}}{T_{1}}$ to 0.001 wich is the most relevant jitter value observed with inverters based oscilators implemented in FPGA's [Ref to be completed]</li>
</ul>
<ul>
<li><p style="color:green">Your goal for this mission is to determine the smallest $D$ for which the distribution and the auto-correlation would not distinguish the EO-TRNG from an ideal RNG for: </p>
    <ul>
      <li>$\alpha_{1}=0.502$ </li>
      <li>$\alpha_{1}=0.498$ </li>
      <li>$\alpha_{1}=0.504$ </li>
      <li>$\alpha_{1}=0.496$ </li>
      <li>$\alpha_{1}=0.506$ </li>
      <li>$\alpha_{1}=0.494$ </li>        
    </ul>
</ul>

### 2<sup>nd</sup>  conclusion
#### <p style="color:red"> How the oscilators duty cycle impact the throughput of a EO-TRNG undistinguishable from an ideal RNG ?</p>


In [1]:
from os import system
#T0=
#T1=
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False    
if IN_COLAB:
    print('NOTEBOOK RUNNING ON COLAB')
    s_cmd='wget https://github.com/patrickhaddadteaching/TRNG_ex1/raw/main/exo1_src.py'
    system(s_cmd)
from exo1_src import *
display(v_box_top)

T0=1.875000e-09 sec.
T1=3.125000e-09 sec.
